# Server

In [24]:
import tekore as tk
from dotenv import load_dotenv
import httpx

load_dotenv()

conf = tk.config_from_environment()
cred = tk.Credentials(*conf)

trans = httpx.HTTPTransport(retries=3)
client = httpx.Client(timeout=30, transport=trans)
sender = tk.SyncSender(client=client)
spotify = tk.Spotify(sender=sender)

host = '127.0.0.1'
port = 5000

In [25]:
from os import environ as env
import json
from authentication.spotify_server import SpotifyServer
try:
    with open(".spotify_credentials.json", "r") as infile:
        token_dict = json.load(infile)
        token_dict["scope"] = " ".join(token_dict["scope"])
        spotify.token = tk.Token(token_dict, token_dict["uses_pkce"])
        spotify.token = cred.refresh(spotify.token)
except Exception as e:
    print(e)
    app = SpotifyServer(host, port, spotify, cred)
    spotify.token = app.spawn_single_use_server()

In [26]:
from typing import List
token: tk.Token = spotify.token
scopes: List[str] = list(token.scope)
token_dict = {
    "token_type": token.token_type,
    "access_token": token.access_token,
    "refresh_token": token.refresh_token,
    "scope": scopes[0].replace("[", "").replace("]", "").replace("'", "").split(", ") if len(scopes) == 0 else scopes,
    "expires_at": token.expires_at,
    "uses_pkce": token.uses_pkce,
    "expires_in": 0
}
json_object = json.dumps(token_dict, indent=4)
with open(".spotify_credentials.json", "w") as outfile:
    outfile.write(json_object)

# Spotify API

In [27]:
spotify.token = cred.refresh(spotify.token)

In [28]:
from project.random_track import RandomTrack

rndTrack = RandomTrack(spotify, cred)
tracks = rndTrack.random_tracks()
[track.name for track in tracks]

['When It Rains, It Pours (feat. Coco, Mr Traumatik & Songer) - Shapes Remix',
 'R3VOLUTION',
 'Lekkere Boterham - GPF Remix',
 'Reignite',
 'Anadrol (Jiyagi Remix)',
 'The Flute Tune - Soulpride Remix',
 "I'm Good (Blue) - REAPER Remix",
 'Groeien - Radio Edit',
 'Lost Control - Jiyagi Remix',
 'Release The Kraken - Sefa Remix',
 'Froz3n - Aftershock Remix',
 'Liberté - Sefa & Outsiders Remix',
 'Died In Your Arms (Reloaded)',
 'Ready To Fly - Sub Focus & Dimension',
 'Getting Started - Chase & Status Remix',
 'RAVE',
 'XYZ - NERO Remix',
 'Remedy',
 'Riptide',
 'Roller',
 "I'm Good (Blue)",
 'Ramenez la coupe à la maison',
 'Rasputin',
 'go - goddard. Remix',
 'Kraut & Ruam',
 'Running Up That Hill (A Deal With God)',
 'The Real Slim Shady',
 'Rich Flex',
 'Run Rudolph Run',
 'LEBENSLANG - HBz Remix',
 'Robbery',
 'R&Drill',
 'Rolling in the Deep',
 'Wonderful Dream (Holidays Are Coming) - Radio Version',
 'Rain In Ibiza',
 'Revenge',
 "Rockin' Around The Christmas Tree",
 "Rockin' A

In [29]:
from project.util import Partition

artist_ids = [artist.id for track in tracks for artist in track.artists]
artist_id_partitions = Partition(artist_ids)
artists = [spotify.artists(ids) for ids in artist_id_partitions]
artists: List[tk.model.FullArtist] = list(artist for partition in artists for artist in partition)
len(artists)

86

In [30]:
import pylast
from authentication.lastfm_credentials import LastFmCredentials
from lastfm import LastFmScraper, LastFmProxy

creds = LastFmCredentials(env["LASTFM_API_KEY"], env["LASTFM_SHARED_SECRET"])
last_network = pylast.LastFMNetwork(
    api_key=creds.api_key,
    api_secret=creds.shared_secret,
)

lastfm = LastFmProxy(last_network, LastFmScraper())

In [31]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

In [32]:
from project.track import TaggedTrack
for track in analyzed_tracks.tracks:
    try:
        tags = TaggedTrack(lastfm, track.name, track.artist_names).tags()
        if len(tags) == 0:
            raise Exception("No tags")
    except Exception as e:
        print(e, track.id, track.name, track.artist_names)
        continue
    track.tags = tags

No tags 0WxwQcXbfVQuVjY7rm4yPd Kraut & Ruam ['Mozhart']
No tags 4FXX4P3dxc1JMBx5wVRJ5N LEBENSLANG - HBz Remix ['Tream', 'HBz', 'treamiboii']


In [33]:
import track as model
analyzed_tracks = model.AnalyzedTracks(tracks, artists)

audio_feature_id_limit=50
track_ids = [track.id for track in analyzed_tracks.tracks]
track_id_partitions = Partition(track_ids)
features = [spotify.tracks_audio_features(ids) for ids in track_id_partitions]
features = [feature for partition in features for feature in partition]

for track in analyzed_tracks.tracks:
    # analysis = spotify.track_audio_analysis(track_id=track.id)
    feature = [feature for feature in features if feature.id == track.id][0]
    track.acousticness = feature.acousticness
    # track.pitches = {segment.start:segment.pitches for segment in analysis.segments}
    track.loudness = feature.loudness
    track.energy = feature.energy
    track.danceability = feature.danceability
    track.mode = feature.mode
    track.instrumentalness = feature.instrumentalness
    track.key = feature.key
    track.liveness = feature.liveness
    track.tempo = feature.tempo
    track.time_signature = feature.time_signature
    track.valence = feature.valence

# Firebase

In [34]:
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

In [37]:
from google.cloud.firestore_v1 import Client

firestore_client: Client = firestore.client()
analyzed_tracks.upsert(firestore_client.collection("tracks"))